In [ ]:
!pip install opencv-python
!pip install numpy

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# Ensure directories exist
os.makedirs('Cropped_Receipts', exist_ok=True)

# ADD YOUR FOLDER PATH
input_folder = 'dataset'
output_folder = 'Cropped_Receipts'

# Gamma correction
invGamma = 1.0 / 0.3
table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")

def biggestRectangle(contours):
    biggest = None
    max_area = 0
    indexReturn = -1
    for index in range(len(contours)):
        i = contours[index]
        area = cv2.contourArea(i)
        if area > 100:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.1 * peri, True)
            if area > max_area:  
                biggest = approx
                max_area = area
                indexReturn = index
    return indexReturn

# Get all image files from input folder
image_files = glob.glob(os.path.join(input_folder, '*.jpg'))

# Process each image
for idx, image_path in enumerate(image_files):
    # Read image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply gamma correction
    gray = cv2.LUT(gray, table)

    # Thresholding
    ret, thresh1 = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Find the biggest rectangle contour
    indexReturn = biggestRectangle(contours)

    if indexReturn != -1:
        # Get the bounding box for the largest contour
        hull = cv2.convexHull(contours[indexReturn])
        x, y, w, h = cv2.boundingRect(hull)
        cv2.drawContours(img, [hull], 0, (0, 255, 0), 3)
        
        # Crop the ROI
        cropped = img[y:y+h, x:x+w]

        # Save cropped image with incremental name
        cropped_filename = os.path.join(output_folder, f'cropped_receipt_{idx+1}.png')
        cv2.imwrite(cropped_filename, cropped)
        print(f"Saved: {cropped_filename}")

        # Optional: Display cropped image
        plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"Cropped Receipt {idx+1}")
        plt.show()
    else:
        print(f"No valid contours found for image: {os.path.basename(image_path)}")
